# Шаг 0. Установка


In [12]:
!pip install vllm faiss-cpu sentence-transformers langchain wikipedia

Exception ignored in: <function tqdm.__del__ at 0x7d54f742a340>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 1147, in __del__
    def __del__(self):

KeyboardInterrupt: 


# Шаг 1. Запускаем vLLM с локальной моделью

In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="Qwen/Qwen2-7B-Instruct-AWQ",
    max_model_len=4096,
)

params = SamplingParams(
    temperature=0,
    max_tokens=300
)

# Шаг 2. Векторизация документов (embeddings)

In [ ]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Шаг 3. Загружаем свои документы и режем их на чанки

In [ ]:
import os
import wikipedia
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

ny = wikipedia.page("New York Sity")

docs = []
docs.extend(text_splitter.split_text(ny.content))

# for fname in os.listdir("docs"):
#     if fname.endswith(".txt"):
#         with open(f"docs/{fname}", "r", encoding="utf8") as f:
#             txt = f.read()
#             docs.extend(text_splitter.split_text(txt))



print("Кусков:", len(docs))


# Шаг 4. Индексируем в FAISS

In [ ]:
import faiss
import numpy as np
from tqdm import tqdm

embs = []
for start in tqdm(range(0, len(docs), 32)):
    batch = docs[start:start+32]
    batch_emb = embedder.encode(batch, device="cuda", convert_to_numpy=True)
    embs.append(batch_emb)

In [ ]:
embs = np.concatenate(embs, axis=0)
dim = embs.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embs)

print("FAISS готов, n=", index.ntotal)

# Шаг 5. Retrieval (поиск релевантных кусков)

In [ ]:
def retrieve(query, k=5):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(q_emb, k)
    return [docs[i] for i in I[0]]

# Шаг 6. RAG prompt → vLLM

In [ ]:
def ask_rag(query):
    passages = retrieve(query)
    context = "\n\n".join(passages)

    prompt = f"""
            You are an assistant, who responds strictly according to context. Context:
            {context}

            Question: {query}
            Response:
            """

    out = llm.generate(prompt, params)
    return out[0].outputs[0].text

In [ ]:
ask_rag("what is the oldest public hospital in the United States?")

# А ТЕПЕРЬ НА РУССКОМ

In [1]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct",
    # model="Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct",
    max_model_len=4096,
)

params = SamplingParams(
    temperature=0,
    max_tokens=80
)

INFO 11-17 17:45:08 [__init__.py:216] Automatically detected platform cuda.
INFO 11-17 17:45:18 [utils.py:233] non-default args: {'max_model_len': 4096, 'disable_log_stats': True, 'model': 'Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 11-17 17:45:19 [model.py:547] Resolved architecture: LlamaForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-17 17:45:19 [model.py:1510] Using max model len 4096
INFO 11-17 17:45:23 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 11-17 17:45:24 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-17 17:45:57 [llm.py:306] Supported_tasks: ['generate']


In [2]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("sergeyzh/rubert-mini-uncased")

In [3]:
# wikipedia.set_lang('ru')

# # 2. Вызываем статью с правильным названием (обратите внимание, что
# #    в русской Википедии статья называется "Нью-Йорк", а не "New York City")
# page = wikipedia.page("Нью-Йорк")

In [4]:
import os
import wikipedia
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

wikipedia.set_lang('ru')
ny = wikipedia.page("Нью-Йорк")

docs = []
docs.extend(text_splitter.split_text(ny.content))

# for fname in os.listdir("docs"):
#     if fname.endswith(".txt"):
#         with open(f"docs/{fname}", "r", encoding="utf8") as f:
#             txt = f.read()
#             docs.extend(text_splitter.split_text(txt))



print("Кусков:", len(docs))

Кусков: 331


In [5]:
import faiss
import numpy as np
from tqdm import tqdm

embs = []
for start in tqdm(range(0, len(docs), 32)):
    batch = docs[start:start+32]
    batch_emb = embedder.encode(batch, device="cuda", convert_to_numpy=True)
    embs.append(batch_emb)

100%|██████████| 11/11 [00:00<00:00, 11.41it/s]


In [6]:
embs = np.concatenate(embs, axis=0)
dim = embs.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embs)

print("FAISS готов, n=", index.ntotal)

FAISS готов, n= 331


In [7]:
def retrieve(query, k=5):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(q_emb, k)
    return [docs[i] for i in I[0]]

In [8]:
def ask_rag(query):
    passages = retrieve(query)
    context = "\n\n".join(passages)

    prompt = f"""
            Ты — ассистент, отвечающий строго по контексту.Контекст:
            {context}

            Вопрос: {query}
            Ответ:
            """

    out = llm.generate(prompt, params)
    return out[0].outputs[0].text

и инференс

In [9]:
def ask_rag_batch(queries, k=2):
    # 1. Batch embedding запросов
    q_embs = embedder.encode(queries, convert_to_numpy=True)

    # 2. Batch поиск в FAISS
    D, I = index.search(q_embs, k)

    # 3. Собираем контексты
    contexts = []
    for ids in I:
        chunks = [docs[i] for i in ids]
        contexts.append("\n\n".join(chunks))

    # 4. Формируем RAG-промпты пачкой
    prompts = [
f"""Ты — ассистент, отвечающий строго по контексту.

Контекст:
{ctx}

Вопрос: {q}
Ответ:"""
        for q, ctx in zip(queries, contexts)
    ]

    # 5. vLLM: batch generation
    outputs = llm.generate(prompts, params)

    # 6. Сбор результатов
    return [
        {"question": q, "answer": out.outputs[0].text.strip()}
        for q, out in zip(queries, outputs)
    ]


In [10]:
questions = [
    "Как называется главный остров, на котором расположена большая часть Нью-Йорка?",
    "Какой район (боро) является самым густонаселенным в Нью-Йорке?",
    "Какое название носил Нью-Йорк, когда он был голландской колонией?",
    "В каком году было завершено строительство Эмпайр-стейт-билдинг?",
    "Сколько всего районов (боро) входит в состав Нью-Йорка? Назовите их.",
    "В каком районе находится Кони-Айленд?",
    "Какой известный мост соединяет районы Манхэттен и Бруклин?",
    "Как называется парк, расположенный в южной части Манхэттена, где находится знаменитый 'Атакующий бык'?",
    "Назовите крупнейший художественный музей в Нью-Йорке, расположенный вдоль 'Музейной мили'.",
    "Какой знаменитый бульвар получил прозвище 'Великий Белый Путь' благодаря своим театрам и световой рекламе?",
    "Как называется площадь, известная своими неоновыми вывесками и являющаяся одной из самых посещаемых достопримечательностей мира?",
    "Где именно расположена Статуя Свободы? Назовите остров.",
    "Как называется знаменитая концертная площадка, известная своей отличной акустикой, где выступали многие легенды музыки?",
    "На какой улице расположена Нью-Йоркская фондовая биржа?",
    "Какое прозвище дал Нью-Йорку писатель О. Генри, которое стало его вторым неофициальным названием?",
    "Где находится здание ООН (Организации Объединенных Наций)?",
    "Какой парк является самым большим и самым известным парком на Манхэттене?",
    "Как называется высокий, заброшенный железнодорожный путь, превращенный в парк на западе Манхэттена?",
    "Какой ботанический сад считается одним из крупнейших в США и расположен в Бронксе?",
    "На какой реке расположены основные районы Нью-Йорка?",
    "Какое из трех главных нью-йоркских аэропортов находится в районе Куинс?",
    "Какого цвета знаменитые нью-йоркские такси?",
    "Какая станция является самой крупной железнодорожной станцией в мире по количеству платформ?",
    "Как называется единственный нью-йоркский район (боро), который не имеет прямой связи метро с Манхэттеном?",
    "Какое животное, согласно нью-йоркской легенде, обитает в туннелях метро?",
    "Какой культовый американский сэндвич считается фирменным блюдом нью-йоркских деликатесных магазинов (деликатесов)?",
    "Какое минимальное количество этажей должно быть в здании, чтобы его считали 'небоскребом' в Нью-Йорке?",
    "Какой день в году считается самым загруженным для парадов в Нью-Йорке, и в честь какого праздника он проводится?",
    "Кто был мэром Нью-Йорка в момент терактов 11 сентября 2001 года?",
    "Какая крупнейшая улица Нью-Йорка проходит с севера на юг и известна своими дорогими магазинами?",
]

In [11]:
batch_size = 5
batched_questions = [questions[i:i+batch_size] for i in range(0, len(questions), batch_size)]

all_results = []
for batch in batched_questions:
    results = ask_rag_batch(batch)
    all_results.extend(results)

for item in all_results:
    print(f"Вопрос: {item['question']}\nОтвет: {item['answer']}\n---")

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

...

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Вопрос: Как называется главный остров, на котором расположена большая часть Нью-Йорка?
Ответ: Манхэттен. Манхэттен — это один из самых крупных и известных островов Нью-Йорка, расположенный в центре города. Он является основным городом Нью-Йорка и является центром политических, экономических и культурныхaktivностей города. Манхэттен известен своими историческими
---
Вопрос: Какой район (боро) является самым густонаселенным в Нью-Йорке?
Ответ: Central Park. Central Park является одним из самых известных парков в мире и является местом отдыха для миллионов жителей Нью-Йорка. Он расположен в центре города и является частью «лёгких Нью-Йорка» — парка, который также включает в себя парки в других районах города. Central Park является
---
Вопрос: Какое название носил Нью-Йорк, когда он был голландской колонией?
Ответ: Название "Нью-Йорк" пришло после того, как город был захвачен англичанами в 1664 году. В то время, когда город был голландской колонией, его называли "Новый Амстердам".
---
Вопр